In [4]:
from huggingface_hub import login
login()

In [5]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from accelerate import dispatch_model
from peft import LoraConfig, get_peft_model, PeftModel

In [3]:
torch.cuda.device_count()

2

In [3]:
!pip install transformers[torch] datasets bitsandbytes trl peft accelerate

  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
  Using cached bitsandbytes-0.43.3-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
  Using cached trl-0.9.6-py3-none-any.whl.metadata (12 kB)
  Using cached peft-0.12.0-py3-none-any.whl.metadata (13 kB)
  Using cached accelerate-0.33.0-py3-none-any.whl.metadata (18 kB)
  Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached aiohttp-3.10.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached tyro-0.8.10-py3-none-any.whl.metadata (8.4 kB)
  Using cached aiohappyeyeballs-2.4.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4

In [4]:
# Load your dataset from Hugging Face
dataset = load_dataset("bhuvana-ak7/mna_training_v3", split="train")

In [5]:
# Load the tokenizer
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"  # Replace with actual path
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [6]:
# Load the packages needed for training
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments

In [7]:
# path where the Trainer will save its checkpoints and logs
trained_model_id = "Llama-3.1-8B-sft-lora-mna-v2"
output_dir = '/home/' + trained_model_id

In [8]:
# Preprocess function
def preprocess_function(examples):
    prompts = [f"### Instruction: {instr}\n### Input: {inp}\n### Output: {out}" 
               for instr, inp, out in zip(examples['instruction'], examples['input'], examples['output'])]
    
    tokenized = tokenizer(prompts, truncation=True, padding="max_length", max_length=4096)
    
    # Create labels (same as input_ids, but with -100 for non-output tokens)
    labels = [[-100] * len(tokenized['input_ids'][i]) for i in range(len(tokenized['input_ids']))]
    for i, prompt in enumerate(prompts):
        output_start = prompt.index("### Output:") + len("### Output:")
        output_tokens = tokenizer(examples['output'][i], add_special_tokens=False)['input_ids']
        labels[i][-len(output_tokens):] = output_tokens
    
    tokenized['labels'] = labels
    return tokenized

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

In [15]:
from datasets import DatasetDict
# Split the dataset into train and test sets (80% train, 20% test)
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

# Create a DatasetDict
dataset_dict = DatasetDict({
    'train': split_dataset['train'],
    'test': split_dataset['test']
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 767
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 192
    })
})


In [9]:
training_args = TrainingArguments(
    fp16=False, # specify bf16=True instead when training on GPUs that support bf16 else fp16
    bf16=False,
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=1,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # originally set to 8
    per_device_train_batch_size=1, # originally set to 8
    push_to_hub=True,
    hub_model_id=trained_model_id,
    report_to="none",
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

In [11]:
from transformers import BitsAndBytesConfig

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
)
device_map = "auto"#{"": torch.cuda.current_device()} if torch.cuda.is_available() else None

model_kwargs = dict(
#     attn_implementation=False,#"flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=quantization_config,
)

In [1]:
# del trainer
torch.cuda.empty_cache()

NameError: name 'torch' is not defined

In [13]:

trainer = SFTTrainer(
        model=model_name,
        model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=split_dataset['train'],
        eval_dataset=split_dataset['test'],
        # dataset_text_field="text",
        tokenizer=tokenizer,
        packing=True,
        peft_config=peft_config,
        max_seq_length=tokenizer.model_max_length,
    )

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': model_init_kwargs, packing, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:155: UserWarning: You passed `model_init_kwargs` to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:185: UserWarning: You passed a model_i

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.


In [14]:
train_result = trainer.train()

***** Running training *****
  Num examples = 959
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 959
  Number of trainable parameters = 54,525,952
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss


ValueError: Trainer: evaluation requires an eval_dataset.

In [16]:
trainer.push_to_hub(private=True)
print('Finished')

Saving model checkpoint to /home/Llama-3.1-8B-sft-lora-mna
loading configuration file config.json from cache at /home/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/snapshots/5206a32e0bd3067aef1ce90f5528ade7d866253f/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_

adapter_model.safetensors:   0%|          | 0.00/218M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/6.07k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

Finished


In [17]:
input = '''
instruction: Extract the following from the input M&A news article: announced_date (The date when the M&A deal was announced), acquiring_company(The name of the acquiring company), acquired_company(The name of the acquired company), deal_amount(The deal amount of the merger or acquisition), mna_type(The type of M&A deal: Acquisition or Merger)
input: 
Mars agrees $36bn deal to buy Pringles maker Kellanova
Mars, the chocolate to pet food group, has struck a $36bn (£28bn) deal to buy Kellanova, the maker of Pringles and Pop-Tarts.
The all-cash offer, announced on Wednesday, is the biggest ever acquisition for the privately owned Mars, dwarfing its $23bn takeover of the chewing gum maker Wrigley in 2008.
The deal could attract scrutiny from competition watchdogs as it brings together well-known consumer goods brands including Mars’ Twix, Bounty and Milky Way with Kellanova’s Pringles, Special K cereal and Carr’s water biscuits.
'''

In [20]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("bhuvana-ak7/Llama-3.1-8B-sft-lora-mna")
base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
model = PeftModel.from_pretrained(base_model, "bhuvana-ak7/Llama-3.1-8B-sft-lora-mna")

adapter_config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/snapshots/5206a32e0bd3067aef1ce90f5528ade7d866253f/config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3.1-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at meta-llama/Meta-Llama-3.1-8B-Instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /home/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/snapshots/5206a32e0bd3067aef1ce90f5528ade7d866253f/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}



adapter_model.safetensors:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [23]:
inputs = tokenizer(input, return_tensors="pt")

In [ ]:
# Generate the output
outputs = model.generate(**inputs, max_new_tokens=100)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
# Decode the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

In [1]:
!git

usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           [--super-prefix=<path>] [--config-env=<name>=<envvar>]
           <command> [<args>]

These are common Git commands used in various situations:

start a working area (see also: git help tutorial)
   clone     Clone a repository into a new directory
   init      Create an empty Git repository or reinitialize an existing one

work on the current change (see also: git help everyday)
   add       Add file contents to the index
   mv        Move or rename a file, a directory, or a symlink
   restore   Restore working tree files
   rm        Remove files from the working tree and from the index

examine the history and state (see also: git help revisions)
   bisect    Use binary search to find th

In [2]:
!git branch

fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [ ]:
https://github.com/bhuvana-ak/huggingface_experiments.git